# Полный анализ датасета Титаник

## Введение

Этот ноутбук содержит **подробный анализ** знаменитого датасета Титаник. Мы проведем:

1. **Исследовательский анализ данных (EDA)**
2. **Визуализацию данных**
3. **Предобработку и очистку данных**
4. **Feature Engineering**
5. **Построение предсказательных моделей**
6. **Оценку и сравнение моделей**

---

## О датасете

**Датасет Титаник** содержит информацию о пассажирах легендарного корабля RMS Titanic, затонувшего 15 апреля 1912 года. Наша цель - предсказать выживаемость пассажиров на основе различных характеристик.

### Основные признаки:

- `PassengerId` - уникальный идентификатор пассажира
- `Survived` - выживаемость (0 = Нет, 1 = Да) - **целевая переменная**
- `Pclass` - класс билета (1 = 1-й класс, 2 = 2-й класс, 3 = 3-й класс)
- `Name` - имя пассажира
- `Sex` - пол
- `Age` - возраст в годах
- `SibSp` - количество братьев/сестер/супругов на борту
- `Parch` - количество родителей/детей на борту
- `Ticket` - номер билета
- `Fare` - стоимость билета
- `Cabin` - номер каюты
- `Embarked` - порт посадки (C = Cherbourg, Q = Queenstown, S = Southampton)

## 1. Импорт необходимых библиотек

Начнем с импорта всех необходимых библиотек для анализа данных, визуализации и машинного обучения.

In [ ]:
# Основные библиотеки для работы с данными
import pandas as pd
import numpy as np

# Визуализация
import matplotlib.pyplot as plt
import seaborn as sns

# Настройка стиля визуализации
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# Для работы с предупреждениями
import warnings
warnings.filterwarnings('ignore')

# Машинное обучение
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, auc

print("Все библиотеки успешно импортированы!")

## 2. Загрузка данных

Загружаем датасет Титаник. Данные можно получить из библиотеки seaborn или напрямую с Kaggle.

In [ ]:
# Загрузка данных из seaborn
df = sns.load_dataset('titanic')

# Создаем копию для сохранения оригинальных данных
df_original = df.copy()

print(f"Размер датасета: {df.shape}")
print(f"Количество строк: {df.shape[0]}")
print(f"Количество признаков: {df.shape[1]}")

## 3. Первичный осмотр данных

Изучим структуру данных и получим общее представление о датасете.

In [ ]:
# Первые строки датасета
print("Первые 10 строк датасета:")
df.head(10)

In [ ]:
# Информация о типах данных и пропущенных значениях
print("Информация о датасете:")
df.info()

In [ ]:
# Статистическое описание числовых признаков
print("Статистическое описание числовых признаков:")
df.describe()

In [ ]:
# Статистика по категориальным признакам
print("Статистика по категориальным признакам:")
df.describe(include=['object', 'category'])

## 4. Анализ пропущенных значений

Пропущенные значения могут существенно влиять на качество моделей. Важно идентифицировать и правильно обработать их.

In [ ]:
# Подсчет пропущенных значений
missing_values = df.isnull().sum()
missing_percent = 100 * df.isnull().sum() / len(df)

missing_df = pd.DataFrame({
    'Признак': missing_values.index,
    'Пропущено значений': missing_values.values,
    'Процент': missing_percent.values
})

missing_df = missing_df[missing_df['Пропущено значений'] > 0].sort_values('Пропущено значений', ascending=False)

print("Пропущенные значения в датасете:")
print(missing_df)

In [ ]:
# Визуализация пропущенных значений
plt.figure(figsize=(12, 6))
sns.heatmap(df.isnull(), cbar=True, cmap='viridis', yticklabels=False)
plt.title('Матрица пропущенных значений', fontsize=16, fontweight='bold')
plt.xlabel('Признаки', fontsize=12)
plt.show()

## 5. Исследовательский анализ данных (EDA)

### 5.1 Анализ целевой переменной (Survived)

Изучим распределение выживших и погибших пассажиров.

In [ ]:
# Подсчет выживших
survival_counts = df['survived'].value_counts()
survival_percent = 100 * df['survived'].value_counts(normalize=True)

print("Распределение выживших:")
print(f"Погибло (0): {survival_counts[0]} ({survival_percent[0]:.2f}%)")
print(f"Выжило (1): {survival_counts[1]} ({survival_percent[1]:.2f}%)")
print(f"\nОбщий уровень выживаемости: {survival_percent[1]:.2f}%")

In [ ]:
# Визуализация
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Столбчатая диаграмма
sns.countplot(data=df, x='survived', ax=axes[0], palette='Set2')
axes[0].set_title('Количество выживших и погибших', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Выживаемость (0 = Погиб, 1 = Выжил)', fontsize=12)
axes[0].set_ylabel('Количество', fontsize=12)

# Добавляем значения на столбцы
for container in axes[0].containers:
    axes[0].bar_label(container)

# Круговая диаграмма
axes[1].pie(survival_counts, labels=['Погибло', 'Выжило'], autopct='%1.1f%%', 
            startangle=90, colors=['#ff9999', '#66b3ff'])
axes[1].set_title('Процентное соотношение выживших', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

### 5.2 Анализ выживаемости по полу

Известно, что при эвакуации приоритет отдавался женщинам и детям. Проверим эту гипотезу.

In [ ]:
# Выживаемость по полу
sex_survival = pd.crosstab(df['sex'], df['survived'], normalize='index') * 100
print("Процент выживаемости по полу:")
print(sex_survival)

# Визуализация
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Сгруппированная столбчатая диаграмма
sex_counts = pd.crosstab(df['sex'], df['survived'])
sex_counts.plot(kind='bar', ax=axes[0], color=['#ff6b6b', '#4ecdc4'])
axes[0].set_title('Выживаемость по полу', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Пол', fontsize=12)
axes[0].set_ylabel('Количество', fontsize=12)
axes[0].set_xticklabels(['Женщины', 'Мужчины'], rotation=0)
axes[0].legend(['Погибло', 'Выжило'])
axes[0].grid(axis='y', alpha=0.3)

# Процентная диаграмма
sex_survival.plot(kind='bar', ax=axes[1], stacked=True, color=['#ff6b6b', '#4ecdc4'])
axes[1].set_title('Процент выживаемости по полу', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Пол', fontsize=12)
axes[1].set_ylabel('Процент', fontsize=12)
axes[1].set_xticklabels(['Женщины', 'Мужчины'], rotation=0)
axes[1].legend(['Погибло', 'Выжило'])
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

### 5.3 Анализ выживаемости по классу билета

Класс билета может отражать социально-экономический статус пассажира и его расположение на корабле.

In [ ]:
# Выживаемость по классу
pclass_survival = pd.crosstab(df['pclass'], df['survived'], normalize='index') * 100
print("Процент выживаемости по классу:")
print(pclass_survival)

# Визуализация
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Сгруппированная столбчатая диаграмма
pclass_counts = pd.crosstab(df['pclass'], df['survived'])
pclass_counts.plot(kind='bar', ax=axes[0], color=['#e74c3c', '#2ecc71'])
axes[0].set_title('Выживаемость по классу билета', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Класс билета', fontsize=12)
axes[0].set_ylabel('Количество', fontsize=12)
axes[0].set_xticklabels(['1-й класс', '2-й класс', '3-й класс'], rotation=0)
axes[0].legend(['Погибло', 'Выжило'])
axes[0].grid(axis='y', alpha=0.3)

# Процентная диаграмма
pclass_survival.plot(kind='bar', ax=axes[1], stacked=True, color=['#e74c3c', '#2ecc71'])
axes[1].set_title('Процент выживаемости по классу билета', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Класс билета', fontsize=12)
axes[1].set_ylabel('Процент', fontsize=12)
axes[1].set_xticklabels(['1-й класс', '2-й класс', '3-й класс'], rotation=0)
axes[1].legend(['Погибло', 'Выжило'])
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

### 5.4 Комбинированный анализ: пол и класс

Посмотрим, как взаимодействуют факторы пола и класса билета.

In [ ]:
# Создаем сводную таблицу
sex_pclass_survival = df.groupby(['sex', 'pclass'])['survived'].mean() * 100
print("Процент выживаемости по полу и классу:")
print(sex_pclass_survival)

# Визуализация
plt.figure(figsize=(12, 6))
sns.barplot(data=df, x='pclass', y='survived', hue='sex', palette='Set1')
plt.title('Выживаемость по классу билета и полу', fontsize=16, fontweight='bold')
plt.xlabel('Класс билета', fontsize=12)
plt.ylabel('Доля выживших', fontsize=12)
plt.legend(title='Пол', labels=['Женщины', 'Мужчины'])
plt.grid(axis='y', alpha=0.3)
plt.show()

### 5.5 Анализ возраста

Изучим распределение возраста пассажиров и его влияние на выживаемость.

In [ ]:
# Статистика по возрасту
print("Статистика по возрасту:")
print(df['age'].describe())
print(f"\nСредний возраст: {df['age'].mean():.2f} лет")
print(f"Медианный возраст: {df['age'].median():.2f} лет")

In [ ]:
# Визуализация распределения возраста
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. Гистограмма возраста
axes[0, 0].hist(df['age'].dropna(), bins=30, color='skyblue', edgecolor='black', alpha=0.7)
axes[0, 0].set_title('Распределение возраста пассажиров', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Возраст', fontsize=12)
axes[0, 0].set_ylabel('Частота', fontsize=12)
axes[0, 0].axvline(df['age'].mean(), color='red', linestyle='--', label=f'Среднее: {df['age'].mean():.1f}')
axes[0, 0].axvline(df['age'].median(), color='green', linestyle='--', label=f'Медиана: {df['age'].median():.1f}')
axes[0, 0].legend()
axes[0, 0].grid(axis='y', alpha=0.3)

# 2. Распределение возраста по выживаемости
df[df['survived']==1]['age'].hist(bins=30, ax=axes[0, 1], color='green', alpha=0.6, label='Выжили')
df[df['survived']==0]['age'].hist(bins=30, ax=axes[0, 1], color='red', alpha=0.6, label='Погибли')
axes[0, 1].set_title('Распределение возраста по выживаемости', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Возраст', fontsize=12)
axes[0, 1].set_ylabel('Частота', fontsize=12)
axes[0, 1].legend()
axes[0, 1].grid(axis='y', alpha=0.3)

# 3. Box plot возраста по выживаемости
sns.boxplot(data=df, x='survived', y='age', ax=axes[1, 0], palette='Set2')
axes[1, 0].set_title('Возраст по выживаемости (Box Plot)', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('Выживаемость (0 = Погиб, 1 = Выжил)', fontsize=12)
axes[1, 0].set_ylabel('Возраст', fontsize=12)

# 4. Violin plot возраста по выживаемости и полу
sns.violinplot(data=df, x='survived', y='age', hue='sex', split=True, ax=axes[1, 1], palette='muted')
axes[1, 1].set_title('Возраст по выживаемости и полу (Violin Plot)', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Выживаемость (0 = Погиб, 1 = Выжил)', fontsize=12)
axes[1, 1].set_ylabel('Возраст', fontsize=12)

plt.tight_layout()
plt.show()

### 5.6 Анализ стоимости билета (Fare)

Стоимость билета также может коррелировать с выживаемостью.

In [ ]:
# Статистика по стоимости билета
print("Статистика по стоимости билета:")
print(df['fare'].describe())
print(f"\nСредняя стоимость: ${df['fare'].mean():.2f}")
print(f"Медианная стоимость: ${df['fare'].median():.2f}")

In [ ]:
# Визуализация
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# 1. Распределение стоимости
axes[0].hist(df['fare'], bins=50, color='coral', edgecolor='black', alpha=0.7)
axes[0].set_title('Распределение стоимости билетов', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Стоимость билета ($)', fontsize=12)
axes[0].set_ylabel('Частота', fontsize=12)
axes[0].set_xlim(0, 300)  # Ограничение для лучшей видимости
axes[0].grid(axis='y', alpha=0.3)

# 2. Box plot стоимости по классу
sns.boxplot(data=df, x='pclass', y='fare', ax=axes[1], palette='Pastel1')
axes[1].set_title('Стоимость билета по классу', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Класс билета', fontsize=12)
axes[1].set_ylabel('Стоимость ($)', fontsize=12)
axes[1].set_ylim(0, 300)

# 3. Box plot стоимости по выживаемости
sns.boxplot(data=df, x='survived', y='fare', ax=axes[2], palette='Set3')
axes[2].set_title('Стоимость билета по выживаемости', fontsize=14, fontweight='bold')
axes[2].set_xlabel('Выживаемость (0 = Погиб, 1 = Выжил)', fontsize=12)
axes[2].set_ylabel('Стоимость ($)', fontsize=12)
axes[2].set_ylim(0, 300)

plt.tight_layout()
plt.show()

### 5.7 Анализ порта посадки (Embarked)

Порт посадки может косвенно указывать на социально-экономический статус.

In [ ]:
# Выживаемость по порту посадки
embarked_survival = pd.crosstab(df['embarked'], df['survived'], normalize='index') * 100
print("Процент выживаемости по порту посадки:")
print(embarked_survival)

# Визуализация
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Количество по портам
embarked_counts = df['embarked'].value_counts()
sns.countplot(data=df, x='embarked', hue='survived', ax=axes[0], palette='viridis')
axes[0].set_title('Выживаемость по порту посадки', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Порт (C=Cherbourg, Q=Queenstown, S=Southampton)', fontsize=12)
axes[0].set_ylabel('Количество', fontsize=12)
axes[0].legend(['Погибло', 'Выжило'])

# Процентная диаграмма
embarked_survival.plot(kind='bar', ax=axes[1], stacked=True, color=['#d62728', '#2ca02c'])
axes[1].set_title('Процент выживаемости по порту посадки', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Порт посадки', fontsize=12)
axes[1].set_ylabel('Процент', fontsize=12)
axes[1].set_xticklabels(['Cherbourg', 'Queenstown', 'Southampton'], rotation=45)
axes[1].legend(['Погибло', 'Выжило'])

plt.tight_layout()
plt.show()

### 5.8 Анализ размера семьи

Создадим новый признак - размер семьи на борту (Family_Size = SibSp + Parch + 1).

In [ ]:
# Создание признака размера семьи
df['family_size'] = df['sibsp'] + df['parch'] + 1

print("Распределение размера семьи:")
print(df['family_size'].value_counts().sort_index())

# Выживаемость по размеру семьи
family_survival = df.groupby('family_size')['survived'].mean()
print("\nВыживаемость по размеру семьи:")
print(family_survival)

In [ ]:
# Визуализация
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Количество по размеру семьи
df['family_size'].value_counts().sort_index().plot(kind='bar', ax=axes[0], color='teal', alpha=0.7)
axes[0].set_title('Распределение размера семьи', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Размер семьи', fontsize=12)
axes[0].set_ylabel('Количество', fontsize=12)
axes[0].set_xticklabels(axes[0].get_xticklabels(), rotation=0)
axes[0].grid(axis='y', alpha=0.3)

# Выживаемость по размеру семьи
family_survival.plot(kind='bar', ax=axes[1], color='orange', alpha=0.7)
axes[1].set_title('Выживаемость по размеру семьи', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Размер семьи', fontsize=12)
axes[1].set_ylabel('Доля выживших', fontsize=12)
axes[1].set_xticklabels(axes[1].get_xticklabels(), rotation=0)
axes[1].axhline(y=df['survived'].mean(), color='r', linestyle='--', label='Средняя выживаемость')
axes[1].legend()
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

### 5.9 Корреляционный анализ

Изучим корреляции между числовыми признаками.

**Коэффициент корреляции Пирсона** измеряет линейную зависимость между двумя переменными:

$$r = \frac{\sum_{i=1}^{n}(x_i - \bar{x})(y_i - \bar{y})}{\sqrt{\sum_{i=1}^{n}(x_i - \bar{x})^2} \sqrt{\sum_{i=1}^{n}(y_i - \bar{y})^2}}$$

где:
- $r \in [-1, 1]$
- $r = 1$: полная положительная корреляция
- $r = -1$: полная отрицательная корреляция
- $r = 0$: отсутствие линейной корреляции

In [ ]:
# Выбираем числовые признаки
numeric_features = ['survived', 'pclass', 'age', 'sibsp', 'parch', 'fare', 'family_size']
correlation_matrix = df[numeric_features].corr()

print("Корреляция признаков с выживаемостью:")
print(correlation_matrix['survived'].sort_values(ascending=False))

In [ ]:
# Визуализация корреляционной матрицы
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
            square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Корреляционная матрица признаков', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

## 6. Предобработка данных

### 6.1 Обработка пропущенных значений

In [ ]:
# Создаем копию для предобработки
df_processed = df.copy()

# 1. Заполнение пропущенных значений Age медианой по группам (пол + класс)
df_processed['age'] = df_processed.groupby(['sex', 'pclass'])['age'].transform(
    lambda x: x.fillna(x.median())
)

# 2. Заполнение Embarked модой (наиболее частым значением)
df_processed['embarked'].fillna(df_processed['embarked'].mode()[0], inplace=True)

# 3. Заполнение Fare медианой
df_processed['fare'].fillna(df_processed['fare'].median(), inplace=True)

# 4. Удаляем признаки с большим количеством пропусков или малой информативностью
df_processed.drop(['deck', 'embark_town'], axis=1, inplace=True, errors='ignore')

print("Пропущенные значения после обработки:")
print(df_processed.isnull().sum())

### 6.2 Feature Engineering

Создадим дополнительные признаки для улучшения предсказательной способности модели.

In [ ]:
# 1. Создаем категориальный признак размера семьи
df_processed['is_alone'] = (df_processed['family_size'] == 1).astype(int)

# 2. Создаем возрастные группы
df_processed['age_group'] = pd.cut(df_processed['age'], 
                                     bins=[0, 12, 18, 35, 60, 100],
                                     labels=['Child', 'Teen', 'Adult', 'Middle', 'Senior'])

# 3. Создаем категории стоимости билета
df_processed['fare_category'] = pd.qcut(df_processed['fare'], 
                                          q=4, 
                                          labels=['Low', 'Medium', 'High', 'Very High'],
                                          duplicates='drop')

# 4. Извлекаем титул из имени
df_processed['title'] = df_processed['name'].str.extract(' ([A-Za-z]+)\.', expand=False)

# Группируем редкие титулы
title_mapping = {
    'Mr': 'Mr', 'Miss': 'Miss', 'Mrs': 'Mrs', 'Master': 'Master',
    'Rev': 'Rare', 'Dr': 'Rare', 'Col': 'Rare', 'Major': 'Rare', 
    'Mlle': 'Miss', 'Mme': 'Mrs', 'Don': 'Rare', 'Dona': 'Rare',
    'Lady': 'Rare', 'Countess': 'Rare', 'Jonkheer': 'Rare', 'Sir': 'Rare',
    'Capt': 'Rare', 'Ms': 'Miss'
}
df_processed['title'] = df_processed['title'].map(title_mapping)
df_processed['title'].fillna('Rare', inplace=True)

print("Новые признаки созданы!")
print(f"\nУникальные титулы: {df_processed['title'].unique()}")
print(f"\nРаспределение по титулам:")
print(df_processed['title'].value_counts())

### 6.3 Кодирование категориальных признаков

In [ ]:
# Создаем копию для моделирования
df_model = df_processed.copy()

# Label Encoding для бинарных признаков
df_model['sex'] = df_model['sex'].map({'male': 0, 'female': 1})

# One-Hot Encoding для категориальных признаков
categorical_features = ['embarked', 'title', 'age_group', 'fare_category']

for feature in categorical_features:
    if feature in df_model.columns:
        dummies = pd.get_dummies(df_model[feature], prefix=feature, drop_first=True)
        df_model = pd.concat([df_model, dummies], axis=1)
        df_model.drop(feature, axis=1, inplace=True)

# Удаляем ненужные признаки
features_to_drop = ['name', 'ticket', 'cabin', 'alive', 'adult_male', 'who', 'alone', 'class']
df_model.drop(features_to_drop, axis=1, inplace=True, errors='ignore')

print("Финальный набор признаков:")
print(df_model.columns.tolist())
print(f"\nРазмер датасета: {df_model.shape}")

## 7. Подготовка данных для моделирования

Разделим данные на обучающую и тестовую выборки.

In [ ]:
# Разделение на признаки (X) и целевую переменную (y)
X = df_model.drop('survived', axis=1)
y = df_model['survived']

# Разделение на обучающую и тестовую выборки (80/20)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Размер обучающей выборки: {X_train.shape}")
print(f"Размер тестовой выборки: {X_test.shape}")
print(f"\nРаспределение классов в обучающей выборке:")
print(y_train.value_counts(normalize=True))
print(f"\nРаспределение классов в тестовой выборке:")
print(y_test.value_counts(normalize=True))

### 7.1 Масштабирование признаков

Некоторые алгоритмы (например, логистическая регрессия, SVM) чувствительны к масштабу признаков.

**Стандартизация (Z-score normalization)**:

$$z = \frac{x - \mu}{\sigma}$$

где $\mu$ - среднее, $\sigma$ - стандартное отклонение.

In [ ]:
# Инициализация scaler
scaler = StandardScaler()

# Масштабирование только на обучающих данных
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Преобразование обратно в DataFrame для удобства
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)

print("Масштабирование признаков завершено!")

## 8. Построение моделей машинного обучения

Обучим несколько различных моделей и сравним их производительность.

### 8.1 Базовая модель - Логистическая регрессия

**Логистическая регрессия** использует логистическую функцию (сигмоиду) для предсказания вероятности класса:

$$P(y=1|x) = \frac{1}{1 + e^{-(\beta_0 + \beta_1x_1 + ... + \beta_nx_n)}}$$

In [ ]:
# Обучение модели
log_reg = LogisticRegression(random_state=42, max_iter=1000)
log_reg.fit(X_train_scaled, y_train)

# Предсказания
y_pred_log = log_reg.predict(X_test_scaled)
y_pred_proba_log = log_reg.predict_proba(X_test_scaled)[:, 1]

# Оценка
acc_log = accuracy_score(y_test, y_pred_log)
print(f"Точность логистической регрессии: {acc_log:.4f}")

# Кросс-валидация
cv_scores_log = cross_val_score(log_reg, X_train_scaled, y_train, cv=5, scoring='accuracy')
print(f"Средняя точность на кросс-валидации: {cv_scores_log.mean():.4f} (+/- {cv_scores_log.std():.4f})")

### 8.2 Дерево решений

In [ ]:
# Обучение модели
dt = DecisionTreeClassifier(random_state=42, max_depth=5)
dt.fit(X_train, y_train)

# Предсказания
y_pred_dt = dt.predict(X_test)
y_pred_proba_dt = dt.predict_proba(X_test)[:, 1]

# Оценка
acc_dt = accuracy_score(y_test, y_pred_dt)
print(f"Точность дерева решений: {acc_dt:.4f}")

# Кросс-валидация
cv_scores_dt = cross_val_score(dt, X_train, y_train, cv=5, scoring='accuracy')
print(f"Средняя точность на кросс-валидации: {cv_scores_dt.mean():.4f} (+/- {cv_scores_dt.std():.4f})")

### 8.3 Random Forest (Случайный лес)

**Random Forest** - это ансамблевый метод, который строит множество деревьев решений и объединяет их предсказания.

Итоговое предсказание для классификации:

$$\hat{y} = \text{mode}\{h_1(x), h_2(x), ..., h_B(x)\}$$

где $h_i(x)$ - предсказание $i$-го дерева, $B$ - количество деревьев.

In [ ]:
# Обучение модели
rf = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=10)
rf.fit(X_train, y_train)

# Предсказания
y_pred_rf = rf.predict(X_test)
y_pred_proba_rf = rf.predict_proba(X_test)[:, 1]

# Оценка
acc_rf = accuracy_score(y_test, y_pred_rf)
print(f"Точность Random Forest: {acc_rf:.4f}")

# Кросс-валидация
cv_scores_rf = cross_val_score(rf, X_train, y_train, cv=5, scoring='accuracy')
print(f"Средняя точность на кросс-валидации: {cv_scores_rf.mean():.4f} (+/- {cv_scores_rf.std():.4f})")

### 8.4 Gradient Boosting

**Gradient Boosting** последовательно обучает модели, где каждая новая модель корректирует ошибки предыдущих.

In [ ]:
# Обучение модели
gb = GradientBoostingClassifier(n_estimators=100, random_state=42, max_depth=3, learning_rate=0.1)
gb.fit(X_train, y_train)

# Предсказания
y_pred_gb = gb.predict(X_test)
y_pred_proba_gb = gb.predict_proba(X_test)[:, 1]

# Оценка
acc_gb = accuracy_score(y_test, y_pred_gb)
print(f"Точность Gradient Boosting: {acc_gb:.4f}")

# Кросс-валидация
cv_scores_gb = cross_val_score(gb, X_train, y_train, cv=5, scoring='accuracy')
print(f"Средняя точность на кросс-валидации: {cv_scores_gb.mean():.4f} (+/- {cv_scores_gb.std():.4f})")

### 8.5 Support Vector Machine (SVM)

In [ ]:
# Обучение модели
svm = SVC(kernel='rbf', random_state=42, probability=True)
svm.fit(X_train_scaled, y_train)

# Предсказания
y_pred_svm = svm.predict(X_test_scaled)
y_pred_proba_svm = svm.predict_proba(X_test_scaled)[:, 1]

# Оценка
acc_svm = accuracy_score(y_test, y_pred_svm)
print(f"Точность SVM: {acc_svm:.4f}")

# Кросс-валидация
cv_scores_svm = cross_val_score(svm, X_train_scaled, y_train, cv=5, scoring='accuracy')
print(f"Средняя точность на кросс-валидации: {cv_scores_svm.mean():.4f} (+/- {cv_scores_svm.std():.4f})")

### 8.6 K-Nearest Neighbors (KNN)

In [ ]:
# Обучение модели
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_scaled, y_train)

# Предсказания
y_pred_knn = knn.predict(X_test_scaled)
y_pred_proba_knn = knn.predict_proba(X_test_scaled)[:, 1]

# Оценка
acc_knn = accuracy_score(y_test, y_pred_knn)
print(f"Точность KNN: {acc_knn:.4f}")

# Кросс-валидация
cv_scores_knn = cross_val_score(knn, X_train_scaled, y_train, cv=5, scoring='accuracy')
print(f"Средняя точность на кросс-валидации: {cv_scores_knn.mean():.4f} (+/- {cv_scores_knn.std():.4f})")

## 9. Сравнение моделей

Сравним производительность всех моделей.

In [ ]:
# Создаем DataFrame с результатами
models_comparison = pd.DataFrame({
    'Модель': ['Logistic Regression', 'Decision Tree', 'Random Forest', 
               'Gradient Boosting', 'SVM', 'KNN'],
    'Точность': [acc_log, acc_dt, acc_rf, acc_gb, acc_svm, acc_knn],
    'CV Средняя': [cv_scores_log.mean(), cv_scores_dt.mean(), cv_scores_rf.mean(),
                   cv_scores_gb.mean(), cv_scores_svm.mean(), cv_scores_knn.mean()],
    'CV Std': [cv_scores_log.std(), cv_scores_dt.std(), cv_scores_rf.std(),
               cv_scores_gb.std(), cv_scores_svm.std(), cv_scores_knn.std()]
})

models_comparison = models_comparison.sort_values('Точность', ascending=False)
print("Сравнение моделей:")
print(models_comparison.to_string(index=False))

In [ ]:
# Визуализация сравнения
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# График точности
colors = plt.cm.viridis(np.linspace(0, 1, len(models_comparison)))
bars1 = axes[0].barh(models_comparison['Модель'], models_comparison['Точность'], color=colors)
axes[0].set_xlabel('Точность', fontsize=12)
axes[0].set_title('Сравнение точности моделей', fontsize=14, fontweight='bold')
axes[0].set_xlim(0.7, 0.9)
axes[0].grid(axis='x', alpha=0.3)

# Добавляем значения на графике
for i, bar in enumerate(bars1):
    width = bar.get_width()
    axes[0].text(width, bar.get_y() + bar.get_height()/2, 
                f'{width:.4f}', ha='left', va='center', fontsize=10)

# График кросс-валидации с доверительными интервалами
axes[1].barh(models_comparison['Модель'], models_comparison['CV Средняя'], 
             xerr=models_comparison['CV Std'], color=colors, capsize=5)
axes[1].set_xlabel('Средняя точность (CV)', fontsize=12)
axes[1].set_title('Точность на кросс-валидации (с std)', fontsize=14, fontweight='bold')
axes[1].set_xlim(0.7, 0.9)
axes[1].grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

## 10. Детальный анализ лучшей модели

Выберем лучшую модель и проведем детальный анализ.

In [ ]:
# Используем Random Forest как одну из лучших моделей
best_model = rf
y_pred_best = y_pred_rf
y_pred_proba_best = y_pred_proba_rf

print("Детальный отчет по классификации:")
print(classification_report(y_test, y_pred_best, target_names=['Погибло', 'Выжило']))

### 10.1 Матрица ошибок (Confusion Matrix)

**Матрица ошибок** показывает количество правильных и неправильных предсказаний:

|                  | Predicted: 0 | Predicted: 1 |
|------------------|--------------|---------------|
| **Actual: 0**    | TN           | FP            |
| **Actual: 1**    | FN           | TP            |

где:
- **TP (True Positive)** - правильно предсказано "выжил"
- **TN (True Negative)** - правильно предсказано "погиб"
- **FP (False Positive)** - ошибочно предсказано "выжил"
- **FN (False Negative)** - ошибочно предсказано "погиб"

**Метрики:**

$$\text{Accuracy} = \frac{TP + TN}{TP + TN + FP + FN}$$

$$\text{Precision} = \frac{TP}{TP + FP}$$

$$\text{Recall} = \frac{TP}{TP + FN}$$

$$\text{F1-Score} = 2 \cdot \frac{\text{Precision} \cdot \text{Recall}}{\text{Precision} + \text{Recall}}$$

In [ ]:
# Матрица ошибок
cm = confusion_matrix(y_test, y_pred_best)

# Визуализация
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=True,
            xticklabels=['Погибло', 'Выжило'],
            yticklabels=['Погибло', 'Выжило'])
plt.title('Матрица ошибок (Confusion Matrix)', fontsize=16, fontweight='bold')
plt.ylabel('Истинное значение', fontsize=12)
plt.xlabel('Предсказанное значение', fontsize=12)
plt.tight_layout()
plt.show()

# Детальная статистика
tn, fp, fn, tp = cm.ravel()
print(f"\nДетальная статистика:")
print(f"True Negatives (TN): {tn}")
print(f"False Positives (FP): {fp}")
print(f"False Negatives (FN): {fn}")
print(f"True Positives (TP): {tp}")
print(f"\nAccuracy: {(tp + tn) / (tp + tn + fp + fn):.4f}")
print(f"Precision: {tp / (tp + fp):.4f}")
print(f"Recall: {tp / (tp + fn):.4f}")

### 10.2 ROC-кривая и AUC

**ROC-кривая** (Receiver Operating Characteristic) показывает соотношение между True Positive Rate и False Positive Rate:

$$\text{TPR (Sensitivity)} = \frac{TP}{TP + FN}$$

$$\text{FPR} = \frac{FP}{FP + TN}$$

**AUC** (Area Under Curve) - площадь под ROC-кривой. Чем ближе к 1, тем лучше модель.

In [ ]:
# Вычисляем ROC-кривую для всех моделей
models_roc = [
    ('Logistic Regression', y_pred_proba_log),
    ('Decision Tree', y_pred_proba_dt),
    ('Random Forest', y_pred_proba_rf),
    ('Gradient Boosting', y_pred_proba_gb),
    ('SVM', y_pred_proba_svm),
    ('KNN', y_pred_proba_knn)
]

plt.figure(figsize=(10, 8))

for model_name, y_proba in models_roc:
    fpr, tpr, _ = roc_curve(y_test, y_proba)
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, lw=2, label=f'{model_name} (AUC = {roc_auc:.3f})')

# Добавляем диагональную линию (случайный классификатор)
plt.plot([0, 1], [0, 1], 'k--', lw=2, label='Случайный классификатор (AUC = 0.5)')

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate (FPR)', fontsize=12)
plt.ylabel('True Positive Rate (TPR)', fontsize=12)
plt.title('ROC-кривые для всех моделей', fontsize=16, fontweight='bold')
plt.legend(loc='lower right', fontsize=10)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

### 10.3 Важность признаков

Для моделей на основе деревьев можем определить важность каждого признака.

In [ ]:
# Получаем важность признаков из Random Forest
feature_importance = pd.DataFrame({
    'Признак': X_train.columns,
    'Важность': rf.feature_importances_
}).sort_values('Важность', ascending=False)

print("Топ-15 наиболее важных признаков:")
print(feature_importance.head(15).to_string(index=False))

In [ ]:
# Визуализация важности признаков
plt.figure(figsize=(12, 8))
top_features = feature_importance.head(15)
colors = plt.cm.viridis(np.linspace(0, 1, len(top_features)))

plt.barh(range(len(top_features)), top_features['Важность'], color=colors)
plt.yticks(range(len(top_features)), top_features['Признак'])
plt.xlabel('Важность признака', fontsize=12)
plt.title('Топ-15 наиболее важных признаков (Random Forest)', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

## 11. Оптимизация гиперпараметров

Используем Grid Search для поиска оптимальных гиперпараметров для Random Forest.

In [ ]:
# Определяем сетку параметров
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, 15, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Grid Search
grid_search = GridSearchCV(
    RandomForestClassifier(random_state=42),
    param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    verbose=1
)

print("Начинаем поиск оптимальных гиперпараметров...")
print("Это может занять некоторое время...")
grid_search.fit(X_train, y_train)

print(f"\nЛучшие параметры: {grid_search.best_params_}")
print(f"Лучшая точность на кросс-валидации: {grid_search.best_score_:.4f}")

In [ ]:
# Оценка оптимизированной модели на тестовой выборке
best_rf = grid_search.best_estimator_
y_pred_optimized = best_rf.predict(X_test)

acc_optimized = accuracy_score(y_test, y_pred_optimized)
print(f"\nТочность оптимизированной модели на тестовой выборке: {acc_optimized:.4f}")
print(f"Улучшение по сравнению с базовой моделью: {(acc_optimized - acc_rf)*100:.2f}%")

print("\nДетальный отчет оптимизированной модели:")
print(classification_report(y_test, y_pred_optimized, target_names=['Погибло', 'Выжило']))

## 12. Выводы и заключение

### Основные выводы из анализа:

1. **Выживаемость по полу**: Женщины имели значительно более высокие шансы на выживание (~74%) по сравнению с мужчинами (~19%), что подтверждает правило "женщины и дети в первую очередь".

2. **Влияние класса билета**: Пассажиры 1-го класса имели шансы на выживание в ~63%, тогда как у пассажиров 3-го класса - только ~24%. Это отражает социально-экономическое неравенство и расположение кают.

3. **Возраст**: Дети имели более высокие шансы на выживание, особенно девочки из высших классов.

4. **Размер семьи**: Пассажиры с небольшими семьями (2-4 человека) имели лучшие шансы на выживание, чем те, кто путешествовал в одиночку или с очень большими семьями.

5. **Производительность моделей**: Наилучшие результаты показали ансамблевые методы (Random Forest и Gradient Boosting) с точностью около 82-85%.

### Наиболее важные признаки для предсказания:
- Пол (sex)
- Класс билета (pclass)
- Стоимость билета (fare)
- Возраст (age)
- Титул (извлеченный из имени)

### Рекомендации для дальнейшего улучшения:
1. Попробовать другие техники feature engineering
2. Использовать более продвинутые методы ансамблирования (Stacking, Blending)
3. Применить современные методы градиентного бустинга (XGBoost, LightGBM, CatBoost)
4. Провести более глубокий анализ взаимодействий между признаками
5. Использовать техники работы с несбалансированными классами

## 13. Сохранение лучшей модели

Сохраним обученную модель для дальнейшего использования.

In [ ]:
import pickle

# Сохранение модели
with open('best_titanic_model.pkl', 'wb') as file:
    pickle.dump(best_rf, file)

# Сохранение scaler
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

print("Модель и scaler успешно сохранены!")
print("Файлы: best_titanic_model.pkl, scaler.pkl")

---

## Конец анализа

Этот ноутбук продемонстрировал полный цикл анализа данных и машинного обучения:
- Исследовательский анализ данных (EDA)
- Визуализацию и понимание данных
- Предобработку и feature engineering
- Построение и сравнение различных моделей
- Оптимизацию гиперпараметров
- Интерпретацию результатов

Вы можете использовать этот ноутбук как шаблон для анализа других датасетов!